#Capstone Project- The Battle of Neighborhoods

    ###Introduction

Our goal is to find the best location in Thessaloniki, Greece where he can open a vegan restaurant.

Veganism is becoming more and more popular nowadays. The reason for these are that more people are becoming health conscious and concerned about ethical eating. 
There are a lot of non-vegans who love vegan food. Those who genuinely love a healthy lifestyle, for instance. So, a good location to open the restaurant is near yoga and pilates studios.
Also we will make sure that we choose a highly visible location and a location not near other vegan restaurants.

###Data

####1.Geographic locations of vegan restaurants of Thessaloniki

In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests


In [3]:
# import functionality to find and load dotenv credentials
# and getenv to get environment variables from OS
from os import getenv
from dotenv import load_dotenv, find_dotenv

# load environment keys, it will automatically find and load .env
load_dotenv(find_dotenv())

True

In [4]:
'''
def login(api_key, secret):
    print('Logged with:')
    print('API: ' + api_id)
    print('SECRET: ' + secret_id)'''

# Logging in to third party app with environment variable credentials
api_id = getenv('CLIENT_ID')
secret_id = getenv('CLIENT_SECRET')
VERSION = '20180605'
#login(api_id, secret_id)

In [5]:
Thessaloniki=[40.63201814602347,22.951599874023486]

In [6]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    api_id, 
    secret_id, 
    VERSION, 
    Thessaloniki[0], 
    Thessaloniki[1], 
    radius, 
    LIMIT)
#url # display URL

In [7]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d752303f96b2c002c3c066d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Thessaloniki',
  'headerFullLocation': 'Thessaloniki',
  'headerLocationGranularity': 'city',
  'totalResults': 243,
  'suggestedBounds': {'ne': {'lat': 40.650018164023486,
    'lng': 22.975273917817358},
   'sw': {'lat': 40.61401812802345, 'lng': 22.927925830229615}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '507eb291e4b04b041ef6c022',
       'name': 'Λόλα',
       'location': {'address': 'Αγαπηνού 10',
        'lat': 40.631781,
        'lng': 22.951488,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.631781,
          'lng': 22.951488}],
     

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [10]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Λόλα,Fish Taverna,40.631781,22.951488
1,Arch of Galerius (Kamara) (Αψίδα Γαλερίου (Καμ...,Monument / Landmark,40.632019,22.951758
2,Coffee Island,Coffee Shop,40.632458,22.951238
3,Tabya,Bar,40.633017,22.954351
4,Valentino,Creperie,40.630545,22.950176


In [11]:
nearby_venues

,name,categories,lat,lng
0,Λόλα,Fish Taverna,40.631781,22.951488
1,Arch of Galerius (Kamara) (Αψίδα Γαλερίου (Καμ...,Monument / Landmark,40.632019,22.951758
2,Coffee Island,Coffee Shop,40.632458,22.951238
3,Tabya,Bar,40.633017,22.954351
4,Valentino,Creperie,40.630545,22.950176
5,ενοχές,Bar,40.630802,22.948992
6,Κρητικό Περιβόλι,Fruit & Vegetable Store,40.631550,22.951176
7,Pulp,Pub,40.632278,22.948048
8,Yok Balık (Γιόκ Μπαλίκ),Souvlaki Shop,40.629676,22.952347
9,ERGON Αγορά,Gourmet Shop,40.631932,22.947611


In [ ]:
categories = {
    'Food': '4d4b7105d754a06374d81259',
    'Vegan/Vegetarian Restaurant':'4bf58dd8d48988d1d3941735',
    'Yoga Studio':'4bf58dd8d48988d102941735',
    'Pilates Studio':'5744ccdfe4b0c0459246b4b2',
    'Weight Loss Center':'590a0744340a5803fd8508c3',
    'Gym':'4bf58dd8d48988d176941735'
}